In [22]:
import json
import pandas as pd
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['local']
col = db['cleaned_data']
source_sub = []

features = []

#class PacketDataa:

def flatten_data(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out



with open("local.traffic_capture_2.json", 'r') as f:
    data = json.load(f)
    print(len(data))
    for j in range(len(data)):
        jsonobj = json.loads(json.dumps(data[j]))
        packetjson = json.loads(jsonobj["pcap"])
        
        for packet in range(len(packetjson)):
            subobj = json.loads(json.dumps(packetjson[packet]['_source']))
            flatobj = {
                "label" : jsonobj["event_type"]
            }
            flatobj.update(flatten_data(subobj))
            for keys in flatobj.keys():
                if keys not in features:
                    features.append(keys)
            col.insert_one(flatobj)


266


In [23]:
my_dict = {'frame':[], 'eth':[],'arp':[],'ip':[],'tcp':[],'udp':[], 'data':[],'rtcp':[],'tls':[],'dhcp':[]}

class feature:
    name: str
    _type: str

    def __init__(self, name, _type):
        self.name = name
        self._type = _type
    def print(self):
        print(f"Name: {self.name}, type: {self._type}")
 
for j in range(len(data)):
        jsonobj = json.loads(data[j]['pcap'])
        for key in source_sub:
            if key in jsonobj[1]['_source']['layers'].keys():
                for sub_key in jsonobj[1]['_source']['layers'][key]:
                    print(type(jsonobj[1]['_source']['layers'][key]))
                    if sub_key not in my_dict[key]:
                        my_dict[key].append(sub_key)
        """for key in jsonobj[1]['_source']['layers'].keys():
            if key not in source_sub:
                source_sub.append(key)
        packets = []
        for i in range(len(jsonobj)):
            packets.append(json.loads(json.dumps(jsonobj[i])))
    
        tempjsonobj = {
            "label" : data[j]['event_type'],
            "packets" : packets
        }
        """
    

In [24]:
total = 0

for key in my_dict:
    for i in range(len(my_dict[key])):
        print(my_dict[key][i])
        
print(total)


0


In [25]:
import pymongoarrow.api as pymongoarrow
from pandas import json_normalize

cursor = col.find()
list_data = list(cursor)
df = json_normalize(list_data)
df = df.drop('_id', axis=1)

working_features = [
    'layers_frame_frame.time_delta',
    'layers_frame_frame.time_relative',
    'layers_frame_frame.len',
    'layers_frame_frame.cap_len',
    'layers_eth_eth.dst_tree_eth.dst.oui',
    'layers_eth_eth.src_tree_eth.src.oui',
    'layers_eth_eth.src_tree_eth.addr.oui',
    'layers_ip_ip.hdr_len',
    'layers_ip_ip.len',
    'layers_ip_ip.ttl',
    'layers_ip_ip.checksum.status',
    'layers_ip_ip.src',
    'layers_ip_ip.addr',
    'layers_ip_ip.src_host',
    'layers_ip_ip.host',
    'layers_ip_ip.dst',
    'layers_ip_ip.dst_host',
    'layers_tcp_tcp.srcport',
    'layers_tcp_tcp.dstport',
    'layers_tcp_tcp.port',
    'layers_tcp_tcp.len',
    'layers_tcp_tcp.seq_raw',
    'layers_tcp_tcp.hdr_len',
    'layers_tcp_tcp.window_size_value',
    'layers_tcp_tcp.analysis_tcp.analysis.bytes_in_flight',
    'layers_tcp_tcp.analysis_tcp.analysis.push_bytes_sent',
    'layers_tls_tls.record_tls.record.content_type',
    'layers_tls_tls.record_tls.record.length'
]

bad_features = ['layers_frame_frame.interface_id_tree_frame.interface_name', 
                'layers_frame_frame.time',
               'layers_frame_frame.time_utc',
               'layers_frame_frame.protocols',
               'layers_eth_eth.dst',
               'layers_eth_eth.dst_tree_eth.addr',
               'layers_eth_eth.src',
               'layers_eth_eth.dst_tree_eth.dst_resolved',
               'layers_eth_eth.dst_tree_eth.dst.oui_resolved',
               'layers_eth_eth.dst_tree_eth.addr_resolved',
               'layers_eth_eth.dst_tree_eth.addr.oui_resolved',
               'layers_eth_eth.src_tree_eth.src_resolved',
                'layers_eth_eth.src_tree_eth.addr',
                'layers_eth_eth.src_tree_eth.addr_resolved',
                'layers_eth_eth.type',
                'layers_ip_ip.dsfield',
                'layers_arp_arp.proto.type',
                'layers_arp_arp.src.hw_mac',
                'layers_arp_arp.dst.hw_mac',
                'layers_eth_eth.src_tree_eth.src.oui_resolved',
                'layers_eth_eth.src_tree_eth.addr_resolved',
                'layers_eth_eth.src_tree_eth.addr.oui_resolved',
                'layers_ip_ip.id',
                'layers_ip_ip.flags',
                'layers_ip_ip.checksum',
                'layers_tcp_tcp.completeness_tree_tcp.completeness.str',
                'layers_tcp_tcp.flags',
                'layers_tcp_tcp.flags_tree_tcp.flags.str',
                'layers_tcp_tcp.checksum',
                'layers_tcp_tcp.payload',
                'layers_tls_tls.record_tls.app_data',
                'layers_tls_tls.record_tls.record.version',
                'layers_tcp_tcp.segment_data'
               ]
ipaddr = [
    'layers_arp_arp.src.proto_ipv4',
    'layers_arp_arp.dst.proto_ipv4',
    'layers_ip_ip.src',
    'layers_ip_ip.addr',
    'layers_ip_ip.src_host',
    'layers_ip_ip.host',
    'layers_ip_ip.dst',
    'layers_ip_ip.dst_host'
]



for ipadd in ipaddr:
    df[ipadd] = df[ipadd].str.replace(r'.', '')




labels = df['label']
features = df.drop('label', axis=1)
workingdf = features[working_features]
features = features.drop(bad_features, axis=1)


In [26]:
#print(workingdf.head())

#print(labels)
search_value = '17:03:03:03:d8:00:00:00:00:00:00:01:ec:c9:41:b5:ef:19:7c:36:5b:c8:ea:0e:09:2d:98:7e:d7:76:a6:b0:e5:e7:3d:c7:a5:cf:7b:2d:ad:94:47:04:2b:10:a9:ad:82:f4:e7:33:f2:32:fa:57:de:95:a8:07:85:a4:e0:f5:1e:78:04:74:e5:b8:6c:7c:d0:48:76:db:f5:0a:8c:aa:5d:49:6f:fa:81:8d:d3:04:46:74:2b:f9:8b:94:5e:67:8a:73:ee:2a:e1:54:54:c5:aa:d2:36:fd:2a:4e:1a:71:0c:0d:65:7c:50:d6:a8:5a:55:f0:1a:5c:cc:76:03:ab:54:d6:c8:98:de:d1:5d:1e:12:d9:9e:64:c0:2c:91:4c:13:38:43:f7:31:18:af:6c:44:35:71:85:fd:39:29:2f:ad:5d:70:7d:72:2b:49:b4:57:0b:1e:15:24:7e:00:13:c6:62:93:cb:f1:ef:12:2c:97:2a:e2:92:57:9d:75:62:37:35:b6:94:bb:12:45:10:0a:42:a0:b2:f2:e2:80:b3:d2:8c:54:d3:24:f6:53:0a:6b:6f:b0:50:34:8d:8e:c6:2c:a3:7c:2e:f1:ee:fc:cc:45:dd:79:7f:d4:33:d2:9d:b1:33:0d:b4:68:28:65:28:cd:aa:47:c9:0b:81:1e:24:3b:36:27:13:5e:00:a4:37:57:08:cd:f8:ed:7d:77:f0:1d:62:f7:0e:bb:15:f9:c8:d8:83:9f:11:44:1b:4d:71:92:d9:f0:a4:d8:f8:b5:0e:79:c3:2a:93:87:27:3b:9c:95:4f:ab:14:eb:53:73:30:b9:57:c6:53:7f:03:92:c1:09:08:a3:7c:1a:20:e6:7b:7f:0c:3a:c4:3d:a5:e2:e6:e5'
cols_with_value = workingdf.columns[workingdf.eq(search_value).any()]
print(cols_with_value)

Index([], dtype='object')


In [38]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print(labels.describe())
X_train, X_test, y_train, y_test = train_test_split(workingdf, labels, test_size=0.3)

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)

print(accuracy_score(y_test, y_hat))

count         21280
unique            3
top       live_view
freq           7840
Name: label, dtype: object
0.9948308270676691


In [34]:
print(labels['label'].describe())

KeyError: 'label'